# OpenZoo ❤️ LangChain

OpenZoo works out-of-the-box with LangChain!

This is the OpenZoo version of the [Langchain Quickstart](https://python.langchain.com/docs/use_cases/chatbots/quickstart/)

---

### Imports & Setup

In [18]:
%pip install langchain-openai
%pip install httpx
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 10.5 MB/s eta 0:00:00
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 18.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

---

### Basic Chat Usage

To instantiate the LLM object using the ChatOpenAI wrapper, just replace 

- 'model_name' with the OpenZoo spec (for example: 'chat')
- 'openai_api_key' with the openzoo api key
- 'openai_api_base' with 'https://api.openzoo.ai/v1'

In [3]:
from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

OPENZOO_API_KEY = os.getenv("OPENZOO_API_KEY")          # The OpenAI client expects the API key to be set in the OPENAI_API_KEY environment variable
OPENZOO_API_BASE = os.getenv("OPENZOO_API_BASE")        # We reset the API base to use the OpenZoo API

chat = ChatOpenAI(
    model_name="chat M", 
    # openai_api_key="tRuesp85Ip4bbyUnN6R33ONHlGyBhqq6", 
    # openai_api_base="https://api.openzoo.ai/v1"
)

Simple chat

In [4]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)

/home/ramshreyas/Documents/Dev/OpenZoo/openzoo-demo/.conda/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content=' How about enjoying a fresh and tangy Caprese salad with ripe tomatoes, creamy mozzarella, and aromatic basil?', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 49, 'total_tokens': 81}, 'model_name': 'chat M', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-cf7b4a2a-e51b-443c-ba4c-07a1aa911f17-0')

---

### Prompt Templates

**Multi-turn chat using prompt templates and memory, using a simple chain**

Create the prompt template and a chain

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

Inference: the model displays its understanding of the context from memory.

In [6]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content=' I apologize for any confusion, as I didn\'t speak earlier. I\'m here to provide a text-based response. I just translated the English sentence "I love programming" into French, which is "J\'adore la programmation."', response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 74, 'total_tokens': 128}, 'model_name': 'chat M', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-bd5ca06f-0778-4635-a3d2-121fd1c8d664-0')

---

### Message history

LangChain has a MessageHistory class which is useful to manage Chat history

In [7]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

Add this message history to the previous chat

In [8]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

AIMessage(content=' The translation of "I love programming" to French is "J\'aime programmer".', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 65, 'total_tokens': 85}, 'model_name': 'chat M', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-aa0e0a6f-d1ce-478b-96f8-8e640425ab3e-0')

Test the ChatBot's memory

In [9]:
demo_ephemeral_chat_history.add_ai_message(response)

demo_ephemeral_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_ephemeral_chat_history.messages})

AIMessage(content=' I said "The translation of \'I love programming\' to French is \'J\'aime programmer\'". I\'m here to assist you with any questions or queries you have, so feel free to ask me anything else you\'d like to know!', response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 101, 'total_tokens': 155}, 'model_name': 'chat M', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-ffe06b75-b8af-4597-a8a6-d3ef47a006d9-0')

---

### Documents as context

LangChain enables you to use documents as well as the message history to form the context for an inference.

Start by installing a vector store (Chroma) and some requirements

In [10]:
%pip install --upgrade --quiet langchain-chroma beautifulsoup4
%pip install google-cloud-aiplatform>=1.38.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, we’ll use a document loader to pull data from a webpage:

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

Next, we split it into smaller chunks that the LLM’s context window can handle and store it in a vector database:

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

Then we embed and store those chunks in a vector database:

In [19]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [21]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(all_splits, embeddings)

Create a retriever to fetch Documents similar to the query

In [22]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("how can langsmith help with testing?")

docs

[Document(page_content='Getting started with LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='Getting started with LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='LangSmith.Self-Hosting: Learn about self-hosting options for LangSmith.Proxy: Learn about the proxy capabilities of LangSmith.Tracing: Learn about the tracing capabilities of LangSmith.Evaluation: Learn about the evaluation capabilities of LangSmith.Prompt Hub Learn about the Prompt Hub, a prompt management tool built into LangSmith.Additional Resources\u200bLangSmith Cookbook: A collection of tutorials and end-to-end walkthroughs using LangSmith.LangChain Python: Docs fo

Creating a Document chain

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

Creating a Retrieval Chain

In [26]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

Test the retrieval

In [28]:
response = retrieval_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

response

{'messages': [HumanMessage(content='how can langsmith help with testing?')],
 'context': [Document(page_content='Getting started with LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | 🦜️🛠️ LangSmith'}),
  Document(page_content='Getting started with LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | 🦜️🛠️ LangSmith'}),
  Document(page_content='LangSmith.Self-Hosting: Learn about self-hosting options for LangSmith.Proxy: Learn about the proxy capabilities of LangSmith.Tracing: Learn about the tracing capabilities of LangSmith.Evaluation: Learn about the evaluation capabilities of LangSmith.Prompt Hub Learn about the Prompt Hub, a prompt management tool built into LangSmith.Additional Resources\u200bLangSmith Cookbook: A co